# Water balance checks
This notebook calculates water balances for all six laugh tests.

## Notes
The Vanderborght case requires a special approach because its outputs are only calculated for a single very long time step. Therefore we need to calculate the balance using the initial conditions as start of the time window and this requires some additional pre-procesing.

## Meta data

| Data  | Value  |
|:---|:---|
| Model name| Structure for Unifying Multiple Modelling Alternatives (SUMMA) |
| Model version  | See attributes in output .nc file |
| Model reference | Clark et al. (2015a,b) |
| Model runs by | R. Zolfaghari |
| Notebook code by | W. Knoben, A. Bennett |

In [1]:
# Modules
from operator import truediv 
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
# Specify the data locations relative to the notebook
file_paths = {'Celia'               : '../lt1_celia1990/output/celia1990_output_timestep.nc',
              'Miller clay'         : '../lt2_miller1998/output/millerClay_output_timestep.nc',
              'Miller loam'         : '../lt2_miller1998/output/millerLoam_output_timestep.nc',
              'Miller sand'         : '../lt2_miller1998/output/millerSand_output_timestep.nc',
              'Vanderborght exp. 1' : ['../lt3_vanderborght2005/output/vanderborght2005_exp1_output_timestep.nc',
                                       '../lt3_vanderborght2005/settingsFiles/summa_zInitialCond_vanderborght2005.nc',
                                       '../lt3_vanderborght2005/settingsFiles/summa_zParamTrial_vanderborght2005_exp1.nc'],
              'Vanderborght exp. 2' : ['../lt3_vanderborght2005/output/vanderborght2005_exp2_output_timestep.nc',
                                       '../lt3_vanderborght2005/settingsFiles/summa_zInitialCond_vanderborght2005.nc',
                                       '../lt3_vanderborght2005/settingsFiles/summa_zParamTrial_vanderborght2005_exp2.nc'],
              'Vanderborght exp. 3' : ['../lt3_vanderborght2005/output/vanderborght2005_exp3_output_timestep.nc',
                                       '../lt3_vanderborght2005/settingsFiles/summa_zInitialCond_vanderborght2005.nc',
                                       '../lt3_vanderborght2005/settingsFiles/summa_zParamTrial_vanderborght2005_exp3.nc'],
              'Wigmosta exp. 1'     : '../lt4_wigmosta1994/output/syntheticHillslope-exp1_output_timestep.nc',
              'Wigmosta exp. 2'     : '../lt4_wigmosta1994/output/syntheticHillslope-exp2_output_timestep.nc',
              'Colbeck exp. 1'      : '../lt5_colbeck1976/output/colbeck1976-exp1_output_timestep.nc',
              'Colbeck exp. 2'      : '../lt5_colbeck1976/output/colbeck1976-exp2_output_timestep.nc',
              'Colbeck exp. 3'      : '../lt5_colbeck1976/output/colbeck1976-exp3_output_timestep.nc',
              'Mizoguchi'           : '../lt6_mizoguchi1990/output/mizoguchi1990_output_timestep.nc'}

In [3]:
# Selection parameters
# We can use these to remove the GRU and HRU dimensions from the output files > easier data handling
GRU, HRU = 0,0

In [4]:
# Function to calculate the water balance components on a given time step
def calc_wb(dat,domain):
       
    # Make some storage variables
    vec_liqError  = []
    vec_state     = []
    vec_stateDiff = []
    
    # Find the timestep size [s]
    dt = round((dat['time'][1] - dat['time'][0]).values/np.timedelta64(1, 's'))
    print('    timestep size = ' + str(dt) + ' s.')

    # Intrinsic densities (needed for snow)
    iden_liq = 1000 # [kg m-3]
    iden_ice = 917
    
    # --- Water balance components soil ---
    # Start of time loop
    for t in range(0,len(dat['time'])-1):
       
        # Specify time as indices
        S_time = t 
        E_time = t+1
    
        # Get the layer variables at t=t
        S_nSoil   = dat['nSoil'].isel(time=S_time).values.astype('int')
        S_nSnow   = dat['nSnow'].isel(time=S_time).values.astype('int')
        S_nLayers = dat['nLayers'].isel(time=S_time).values.astype('int')
        
        # Do computations based on the specified domain
        if domain == 'snow':
            
            # Snow layer depths
            S_mLayerDepth = dat['mLayerDepth'].isel(time=S_time,midToto=slice(0,S_nSnow)).values # needs to start at index 0, not 1 in Python
            
            # Snow water at the start of t=t
            S_mLayerVolFraqLiq = dat['mLayerVolFracLiq'].isel(time=S_time,midToto=slice(0,S_nSnow)).values
            S_mLayerVolFraqIce = dat['mLayerVolFracIce'].isel(time=S_time,midToto=slice(0,S_nSnow)).values
            mass0 = sum( (S_mLayerVolFraqLiq * iden_liq + S_mLayerVolFraqIce * iden_ice) * S_mLayerDepth )
            snowBalance0 = mass0 / iden_liq

            # Layer variables at t=t+1
            E_nSoil   = dat['nSoil'].isel(time=E_time).values.astype('int')
            E_nSnow   = dat['nSnow'].isel(time=E_time).values.astype('int')
            E_nLayers = dat['nLayers'].isel(time=E_time).values.astype('int')
            E_mLayerDepth = dat['mLayerDepth'].isel(time=E_time,midToto=slice(0,E_nSnow)).values # needs to start at index 0, not 1 in Python

            # Rainfall flux between t=t and t=t+1
            scalarRainfall = (dat['scalarRainfall'].isel(time=E_time).values / iden_liq) * dt # [kg m-2 s-1] / [kg m-3] * [s] = [m]
            
            # Snowfall flux between t=t and t=t+1
            scalarSnowfall = (dat['scalarSnowfall'].isel(time=E_time).values / iden_ice) * dt # [kg m-2 s-1] / [kg m-3] * [s] = [m]
            
            # Melt flux between t=t and t=t+1
            scalarRainPlusMelt = dat['scalarRainPlusMelt'].isel(time=E_time).values * dt # [m s-1] * [s] = [m]
            
            # Snow water at the end of t=t; i.e. at the start of t=t+1
            E_mLayerVolFraqLiq = dat['mLayerVolFracLiq'].isel(time=E_time,midToto=slice(0,S_nSnow)).values
            E_mLayerVolFraqIce = dat['mLayerVolFracIce'].isel(time=E_time,midToto=slice(0,S_nSnow)).values
            mass1 = sum( (E_mLayerVolFraqLiq * iden_liq + E_mLayerVolFraqIce * iden_ice) * E_mLayerDepth )
            snowBalance1 = mass1 / iden_liq
            
            # Water balance error
            liqError = snowBalance1 - (snowBalance0 + scalarRainfall + scalarSnowfall - scalarRainPlusMelt)
            
            # Append
            vec_liqError.append(liqError)
            vec_state.append(snowBalance1)
            vec_stateDiff.append(snowBalance1 - snowBalance0)
        
        elif domain == 'soil':
        
            # Soil layer depths
            S_mLayerDepth = dat['mLayerDepth'].isel(time=S_time,midToto=slice(S_nSnow,S_nLayers)).values # needs to start at index 0, not 1 in Python
        
            # Soil water at the start of t=t
            S_mLayerVolFraqLiq = dat['mLayerVolFracLiq'].isel(time=S_time,midToto=slice(S_nSnow,S_nLayers)).values
            S_mLayerVolFraqIce = dat['mLayerVolFracIce'].isel(time=S_time,midToto=slice(S_nSnow,S_nLayers)).values
            soilBalance0 = sum( (S_mLayerVolFraqLiq + S_mLayerVolFraqIce) * S_mLayerDepth )   
    
            # Layer variables at t=t+1
            E_nSoil   = dat['nSoil'].isel(time=E_time).values.astype('int')
            E_nSnow   = dat['nSnow'].isel(time=E_time).values.astype('int')
            E_nLayers = dat['nLayers'].isel(time=E_time).values.astype('int')
            E_mLayerDepth = dat['mLayerDepth'].isel(time=E_time,midToto=slice(E_nSnow,E_nLayers)).values # needs to start at index 0, not 1 in Python
        
            # Infiltration between t=t and t=t+1 (needs t=t+1)
            inFlux = dat['scalarInfiltration'].isel(time=E_time).values*dt
    
            # Transpiration between t=t and t=t+1 (needs t=t+1)
            tranSink = dat['mLayerTranspire'].isel(time=E_time).sum().values*dt 
    
            # Baseflow between t=t and t=t+1 (needs t=t+1)
            baseSink = dat['mLayerBaseflow'].isel(time=E_time).sum().values*dt
    
            # Soil drainage between t=t and t=t+1 (needs t=t+1)
            drainage = dat['scalarSoilDrainage'].isel(time=E_time).values*dt
    
            # Compression between t=t and t=t+1 (needs t=t+1)
            mLayerCompress = dat['mLayerCompress'].isel(time=E_time,midSoil=slice(0,E_nSoil)).values
            compSink = sum(mLayerCompress * E_mLayerDepth)
    
            # Soil water at the end of t=t; i.e. at the start of t=t+1
            E_mLayerVolFraqLiq = dat['mLayerVolFracLiq'].isel(time=E_time,midToto=slice(E_nSnow,E_nLayers)).values
            E_mLayerVolFraqIce = dat['mLayerVolFracIce'].isel(time=E_time,midToto=slice(E_nSnow,E_nLayers)).values
            soilBalance1 = sum( (E_mLayerVolFraqLiq + E_mLayerVolFraqIce) * E_mLayerDepth )
    
            # Water balance error
            liqError = soilBalance1 - (soilBalance0 + inFlux + tranSink - drainage - baseSink - compSink)
                
            # Append
            vec_liqError.append(liqError)
            vec_state.append(soilBalance1)
            vec_stateDiff.append(soilBalance1 - soilBalance0)
            
    return vec_liqError, vec_state, vec_stateDiff

In [5]:
def prep_vanderborght(file,HRU,GRU):
    
    # Unpack the files for clarity
    file_sim = file[0] # simulations
    file_ics = file[1] # initial conditions
    file_par = file[2] # parameters
    
    # load the data
    dat = xr.open_dataset( file_sim ).isel(hru=HRU, gru=GRU).load()
    ics = xr.open_dataset( file_ics ).isel(hru=HRU) # Has no GRU dimension
    par = xr.open_dataset( file_par )
    
    # --- Update the mLayerVolFracLiq values (we know ice content is 0) ---
    psi = ics['mLayerMatricHead'].values
    ics['mLayerVolFracLiq'][:] = calc_volFracLiq(psi,par)
    
    # --- Time settings for IC file ---
    # Find the datetime at the start of the simulation timestep
    timeE = dat['time'].values #timestamp at the end of the simulations
    timeD = 86400000 # [s] the Vanderborght experiments are run using a single (huge) timestep of 1000 days
    timeS = timeE - np.timedelta64(timeD,'s') #timestamp of the start of the timestep
    
    # Assign a time dimension to the ICs for merging
    icsn = ics.expand_dims(time=timeS)
    
    # --- Merge initial conditions and simulations ---
    # Variables with a 'midToto' dimension
    var = ['mLayerDepth','mLayerVolFracLiq','mLayerVolFracIce']
    merged = xr.merge([dat[var].where(dat[var] != -9999, drop=True), \
                        icsn[var]])

    # Variables that are not part of the initial conditions and can be merged easily
    #var = ['nSoil','nSnow','nLayers','iLayerLiqFluxSoil','mLayerTranspire','mLayerCompress','mLayerBaseflow']
    var = ['nSoil','nSnow','nLayers','scalarInfiltration','scalarSoilDrainage','scalarSoilBaseflow', \
       'mLayerBaseflow','mLayerTranspire','mLayerCompress','iLayerLiqFluxSoil']
    for v in var:
        merged[v] = dat[v]
    
    # Fill the "nX" variables on first time step
    var = ['nSoil','nSnow']
    for v in var:
        merged[v].loc[dict(time=timeS)] = icsn[v].isel(scalarv=0).values
    merged['nLayers'].loc[dict(time=timeS)] = merged['nSoil'].loc[dict(time=timeS)].values + \
                                                merged['nSnow'].loc[dict(time=timeS)]
       
    return merged

In [6]:
def calc_volFracLiq(psi,pars):
    # Computes the volumetric liquid water content given psi and soil hydraulic parameters theta_res, 
    # theta_sat, alpha, n, and m.
    # See summa/build/source/engine/soil_utils.f90; line 282-300
    
    # Get parameter values
    theta_sat = pars['theta_sat'].values.flatten()
    theta_res = pars['theta_res'].values.flatten()
    alpha     = pars['vGn_alpha'].values.flatten()
    n         = pars['vGn_n'].values.flatten()
    m         = 1 - 1/n
    
    # Compute volumetric liquid fraction
    volFracLiq = theta_res + (theta_sat - theta_res) * (1 + (alpha * psi)**n)**(-1*m)
       
    # Cap volFracLiq at theta_res where appropriate
    volFracLiq[psi >= 0] =  theta_res[psi >= 0]    
    
    return volFracLiq

In [7]:
def calc_relErrSeries(wbe,dStates,states):
    
    # Convert to Numpy arrays for easier indexing
    wbe = np.array(wbe)
    dStates = np.array(dStates)
    states = np.array(states)
    
    # Calculate the relative error series
    rn = list(map(truediv,  wbe[np.nonzero(dStates)[0]], dStates[np.nonzero(dStates)[0]])) # divide two lists element-wise
    rrn = list(map(truediv, wbe[np.nonzero(states)[0]],   states[np.nonzero(states)[0]])) 
        
    return rn, rrn

Processing starts here.

In [17]:
calculateRelativeMetrics = False

In [18]:
# Initialize some lists
test_name   = []
maxAbsErr   = []
meanAbsErr  = []
cumAbsErr   = []
maxRelErr1  = []
meanRelErr1 = []
maxRelErr2  = []
meanRelErr2 = []

# Loop over the files
for test,file in file_paths.items():
    
    # Progress
    print('Working on ' + test)
    
    # Load the data
    if 'Colbeck' in test:
        dat = xr.open_dataset( file ).isel(hru=HRU).load() # Colbeck output has no GRU dimensions
        domain = 'snow'
    elif 'Vanderborght' in test:
        dat = prep_vanderborght( file,HRU,GRU ) # Vanderborght needs special data prep; 'file' contains sim, initial conditions and parameters
        domain = 'soil'
    else: # Celia, Miller, Wigmosta, Mizoguchi
        dat = xr.open_dataset( file ).isel(hru=HRU, gru=GRU).load() 
        domain = 'soil'
    
    # Get water balance values
    wbe,states,dStates = calc_wb(dat,domain)

    # Calculate metrics
    test_name.append(test)
    maxAbsErr.append(np.max(np.abs(wbe)))
    meanAbsErr.append(np.mean(np.abs(wbe)))
    cumAbsErr.append(np.sum(np.abs(wbe)))
    
    # Calculate the relative error metrics if requested
    if calculateRelativeMetrics:
        rn, rrn = calc_relErrSeries(wbe,dStates,states)
        maxRelErr1.append(np.max(rn))
        meanRelErr1.append(np.mean(rn))      
        maxRelErr2.append(np.max(rrn))
        meanRelErr2.append(np.mean(rrn))

Working on Celia
    timestep size = 1800.0 s.
Working on Miller clay
    timestep size = 900.0 s.
Working on Miller loam
    timestep size = 900.0 s.
Working on Miller sand
    timestep size = 900.0 s.
Working on Vanderborght exp. 1
    timestep size = 86400000.0 s.
Working on Vanderborght exp. 2
    timestep size = 86400000.0 s.
Working on Vanderborght exp. 3
    timestep size = 86400000.0 s.
Working on Wigmosta exp. 1
    timestep size = 3600.0 s.
Working on Wigmosta exp. 2
    timestep size = 3600.0 s.
Working on Colbeck exp. 1
    timestep size = 60.0 s.
Working on Colbeck exp. 2
    timestep size = 60.0 s.
Working on Colbeck exp. 3
    timestep size = 60.0 s.
Working on Mizoguchi
    timestep size = 60.0 s.


In [19]:
# Make a dataframe
if not calculateRelativeMetrics:
    results = pd.DataFrame( {'Test'               : test_name,
                             'Max. abs. err. [m]' : maxAbsErr,
                             'Mean abs. err. [m]' : meanAbsErr,
                             'Cum. abs. err. [m]' : cumAbsErr},
                           columns = ['Test', \
                                      'Max. abs. err. [m]', \
                                      'Mean abs. err. [m]', \
                                      'Cum. abs. err. [m]'])
    
else:
    results = pd.DataFrame( {'Test'               : test_name,
                             'Max. abs. err.   [m]' : maxAbsErr,
                             'Mean abs. err.   [m]' : meanAbsErr,
                             'Cum. abs. err.   [m]' : cumAbsErr,
                             'Max. rel. err. 1 [-]' : maxRelErr1,
                             'Mean rel. err. 1 [-]' : meanRelErr1,
                             'Max. rel. err. 2 [-]' : maxRelErr2,
                             'Mean rel. err. 2 [-]' : meanRelErr2},
                           columns = ['Test', \
                                      'Max. abs. err.   [m]', \
                                      'Mean abs. err.   [m]', \
                                      'Cum. abs. err.   [m]', \
                                      'Max. rel. err. 1 [-]', \
                                      'Mean rel. err. 1 [-]', \
                                      'Max. rel. err. 2 [-]', \
                                      'Mean rel. err. 2 [-]'])

results

,Test,Max. abs. err. [m],Mean abs. err. [m],Cum. abs. err. [m]
0,Celia,8.187895e-16,1.902073e-16,2.263467e-14
1,Miller clay,1.424614e-06,3.792766e-08,9.026783e-06
2,Miller loam,5.329071e-15,2.332401e-15,5.551115e-13
3,Miller sand,1.427400e-10,3.288711e-12,7.827132e-10
4,Vanderborght exp. 1,6.149307e-09,6.149307e-09,6.149307e-09
5,Vanderborght exp. 2,9.910798e-09,9.910798e-09,9.910798e-09
6,Vanderborght exp. 3,1.165677e-08,1.165677e-08,1.165677e-08
7,Wigmosta exp. 1,4.725063e-05,9.416612e-06,9.473112e-03
8,Wigmosta exp. 2,5.402572e-05,1.381756e-05,1.390047e-02
9,Colbeck exp. 1,1.221523e-12,1.267628e-14,7.593093e-12
